In [204]:
cd "nilm data\all"

[WinError 3] The system cannot find the path specified: 'nilm data\\all'
D:\jjenko\nilm data\all


In [205]:
from nilmtk import DataSet
from nilmtk.utils import print_dict
from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nilmtk


from nilmtk import DataSet, MeterGroup
import h5py
import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField
from pyts.image import RecurrencePlot

plt.style.use('ggplot')
rcParams['figure.figsize'] = (12, 10)
plt.figure(figsize=(10,30))

#dataset = DataSet('refit.h5')

refit = DataSet('refit.h5')
ukdale = DataSet('ukdale.h5')
eco = DataSet('eco.h5')
redd = DataSet('redd.h5')
iawe = DataSet('iawe.h5')

#dataset.set_window(start='2011-1-17T1800',end='2011-9-18T1900')
#dataset.set_window(start='2012-9-17T1800',end='2012-11-18T1900')b
#manualy_selected_appliances = ['freezer','washer dryer','washing machine','dish washer','computer','television','electric space heater']
#date_range = pd.date_range("2013-10-10T0000", "2015-8-20T0000",freq=f'{step_in_mins}''T') #max 2014-9


<Figure size 720x2160 with 0 Axes>

In [71]:
refit.set_window("2013-10-10T0000", "2015-8-20T0000")
elec = refit.buildings[1].elec
appl = elec.select_using_appliances(type='freezer',instance=1)


#appl.available_columns()
df = next(appl.load(physical_quantity='power'))
 #df = df.power.active #for iawe
fig, ax = plt.subplots()
ax.plot(df.fillna(0))

ax.set(xlabel='time (min)', ylabel='P', title='P')
ax.grid()

#df.power.active[1900:1950]

Loading data for meter ElecMeterID(instance=2, building=1, dataset='REFIT')     

KeyboardInterrupt: 

In [3]:
def printLog(*args, **kwargs):
    #print(*args, **kwargs)
    with open('D:/jjenko/nilm data/GAF_DS/uptime_parser_output.txt','a') as file:
        print(*args, **kwargs, file=file)

In [213]:
datasets = [refit, eco, redd, iawe, ukdale]
printLog("starting parsing uptime.....","\n")
printLog("data_list = [","\n")

for dataset in datasets:
    n_buildings = len(dataset.buildings)

    for building in range(n_buildings):
        building+=1
        
        for meter in dataset.buildings[building].elec.submeters().meters:
            
            appliance_metadata = meter.appliances[0].metadata

            label = appliance_metadata.get("type")
            instance = appliance_metadata.get("instance")

            print("dataset: ",dataset.metadata["name"]," building: ",building," label: ",label," instance: ",instance, "")
            #printLog("dataset: ",dataset.metadata["name"]," building: ",building," label: ",label," instance: ",instance, "")
            
            print("calulating length... ")
            #printLog("calulating length... ")
            df = next(meter.load(physical_quantity='power'))
            ts = df.power.values.transpose()[0]
            ts_on = ts[ts > 20]
            ts_on_len = len(ts_on)
            print("length",ts_on_len)
            #printLog("length",ts_on_len)

            data_list = [dataset.metadata["name"],building,label,instance,ts_on_len]
            printLog(data_list,",","\n")
            

printLog("]","\n")
            

dataset:  REFIT  building:  1  label:  fridge  instance:  1 
calulating length... 
length 1574945
dataset:  REFIT  building:  1  label:  freezer  instance:  1 
calulating length... 
length 2387436
dataset:  REFIT  building:  1  label:  freezer  instance:  2 
calulating length... 
length 2788429
dataset:  REFIT  building:  1  label:  washer dryer  instance:  1 
calulating length... 
length 18377
dataset:  REFIT  building:  1  label:  washing machine  instance:  1 
calulating length... 
length 114389
dataset:  REFIT  building:  1  label:  dish washer  instance:  1 
calulating length... 
length 51661
dataset:  REFIT  building:  1  label:  computer  instance:  1 
calulating length... 
length 569089
dataset:  REFIT  building:  1  label:  television  instance:  1 
calulating length... 
length 1203849
dataset:  REFIT  building:  1  label:  electric space heater  instance:  1 
calulating length... 
length 471432
dataset:  REFIT  building:  2  label:  fridge freezer  instance:  1 
calulating le

In [13]:
refit.buildings[1].elec.submeters().meters

[ElecMeter(instance=2, building=1, dataset='REFIT', appliances=[Appliance(type='fridge', instance=1)]),
 ElecMeter(instance=3, building=1, dataset='REFIT', appliances=[Appliance(type='freezer', instance=1)]),
 ElecMeter(instance=4, building=1, dataset='REFIT', appliances=[Appliance(type='freezer', instance=2)]),
 ElecMeter(instance=5, building=1, dataset='REFIT', appliances=[Appliance(type='washer dryer', instance=1)]),
 ElecMeter(instance=6, building=1, dataset='REFIT', appliances=[Appliance(type='washing machine', instance=1)]),
 ElecMeter(instance=7, building=1, dataset='REFIT', appliances=[Appliance(type='dish washer', instance=1)]),
 ElecMeter(instance=8, building=1, dataset='REFIT', appliances=[Appliance(type='computer', instance=1)]),
 ElecMeter(instance=9, building=1, dataset='REFIT', appliances=[Appliance(type='television', instance=1)]),
 ElecMeter(instance=10, building=1, dataset='REFIT', appliances=[Appliance(type='electric space heater', instance=1)])]

In [35]:

elec = refit.buildings[2].elec
appliance_data = elec["fridge"]

In [30]:
elec.appliances[1]

[Appliance(type='electric space heater', instance=1),
 Appliance(type='freezer', instance=1),
 Appliance(type='dish washer', instance=1),
 Appliance(type='washing machine', instance=1),
 Appliance(type='washer dryer', instance=1),
 Appliance(type='freezer', instance=2),
 Appliance(type='fridge', instance=1),
 Appliance(type='computer', instance=1),
 Appliance(type='television', instance=1)]

In [210]:
meters = refit.buildings[1].elec.submeters().meters


In [20]:
labels = []
for meter in meters:
    appliance_metadata = meter.appliances[0].metadata
    labels.append(appliance_metadata.get("type"))
    

NameError: name 'meters' is not defined

In [69]:
labels

['fridge',
 'freezer',
 'freezer',
 'washer dryer',
 'washing machine',
 'dish washer',
 'computer',
 'television',
 'electric space heater']

In [209]:
eco.buildings[1].elec["fridge",2]

KeyError: {'type': 'fridge', 'instance': 2}

In [177]:
appl[0].label()

"('freezer', 1)"

In [127]:
refit.buildings[1].elec.submeters().meters[3]

ElecMeter(instance=5, building=1, dataset='REFIT', appliances=[Appliance(type='washer dryer', instance=1)])

In [218]:
appliances = []
for sublist in data_list:
    appliances.append(sublist[2])

appliance_set = set(appliances)
appliances_unique = list(appliance_set)

In [241]:
df = pd.DataFrame(data_list)

In [78]:
sum(df.loc[df[2]=="fridge"][4])

19275041

In [221]:
num_instances = []
acc_lenght = []
for appliance in appliance_set:
    num_instances.append(sum(sublist.count(appliance) for sublist in data_list))
    acc_lenght.append(sum(df.loc[df[2]==appliance][4]))
   

In [222]:
print("parsed_data = [")
for row in range(len(appliances_unique)):
    print("['",appliances_unique[row],"',",num_instances[row],",",acc_lenght[row],"],")
    

print("]")

In [226]:
df2 = pd.DataFrame(parsed_data)

In [234]:
pd.set_option('display.max_rows', None)

In [222]:
print("parsed_data = [")
for row in range(len(appliances_unique)):
    print("['",appliances_unique[row],"',",num_instances[row],",",acc_lenght[row],"],")
    

print("]")

In [222]:
print("parsed_data = [")
for row in range(len(appliances_unique)):
    print("['",appliances_unique[row],"',",num_instances[row],",",acc_lenght[row],"],")
    

print("]")

In [236]:
df2.sort_values(by=2,ascending=False)

,0,1,2
9,freezer,18,66054453
3,fridge freezer,17,44222574
69,fridge,20,42416005
46,television,25,37973296
60,computer,18,35179868
22,HTPC,6,23379655
11,light,34,12133088
21,laptop computer,9,7807734
28,audio system,7,6944925
10,microwave,26,6237136


In [242]:
datasets =df[0].unique()

In [243]:
dataset_names = []
leng = []
n_build = []

for dataset in datasets:
    dataset_names.append(dataset)
    leng.append(df.loc[df[0]==dataset][4].sum())
    n_build.append(df.loc[df[0]==dataset][1].max())
    
    
df_dataset = pd.DataFrame({"names":dataset_names, "leng":leng,"n_build":n_build})

In [244]:
df_dataset.sort_values(by="leng",ascending=False)

,names,leng,n_build
0,REFIT,144341850,20
1,ECO,130187888,6
4,UK-DALE,54997820,5
3,iAWE,6425055,1
2,REDD,5890953,6
